<a href="https://colab.research.google.com/github/jasrajs/waste-classification/blob/main/jasraj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

# New Section

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
%cd "/content/drive/MyDrive/Garbage classification"

/content/drive/MyDrive/Garbage classification


In [10]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gen_train = ImageDataGenerator(rescale = 1/255, shear_range = 0.2, zoom_range = 0.2, 
                               brightness_range = (0.1, 0.5), horizontal_flip=True)

train_data = gen_train.flow_from_directory("/content/drive/MyDrive/Garbage classification",
                                           target_size = (224, 224), batch_size = 32, class_mode="categorical")

Found 2527 images belonging to 6 classes.


In [12]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg16 = VGG16(input_shape = (224, 224, 3), weights = "imagenet", include_top = False)

In [13]:
for layer in vgg16.layers:
  layer.trainable = False

In [14]:
from tensorflow.keras import layers

x = layers.Flatten()(vgg16.output)

In [15]:
prediction = layers.Dense(units = 6, activation="softmax")(x)

# creating a model object

model = tf.keras.models.Model(inputs = vgg16.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics =["accuracy"])

result = model.fit_generator(train_data, epochs = 28, steps_per_epoch=len(train_data))

<ipython-input-16-f2c6a01324d2>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  result = model.fit_generator(train_data, epochs = 28, steps_per_epoch=len(train_data))


Epoch 1/28
79/79 [==============================] - 582s 7s/step - loss: 1.6278 - accuracy: 0.3870
Epoch 2/28
79/79 [==============================] - 43s 543ms/step - loss: 1.2209 - accuracy: 0.5362
Epoch 3/28
79/79 [==============================] - 43s 541ms/step - loss: 1.1422 - accuracy: 0.5524
Epoch 4/28
79/79 [==============================] - 42s 528ms/step - loss: 1.1020 - accuracy: 0.5809
Epoch 5/28
79/79 [==============================] - 42s 530ms/step - loss: 1.0154 - accuracy: 0.6090
Epoch 6/28
79/79 [==============================] - 42s 530ms/step - loss: 1.0012 - accuracy: 0.6138
Epoch 7/28
79/79 [==============================] - 42s 528ms/step - loss: 0.9656 - accuracy: 0.6280
Epoch 8/28
79/79 [==============================] - 42s 526ms/step - loss: 0.8896 - accuracy: 0.6712
Epoch 9/28
79/79 [==============================] - 42s 533ms/step - loss: 0.9048 - accuracy: 0.6565
Epoch 10/28
79/79 [==============================] - 42s 533ms/step - loss: 0.8552 - accuracy

In [17]:
from tensorflow.keras.preprocessing import image
output_class = ["cardboard", "glass", "trash", "metal", "paper", "plastic"]
def waste_prediction(new_image):
  test_image = image.load_img(new_image, target_size = (224,224))
  plt.axis("off")
  plt.imshow(test_image)
  plt.show()
 
  test_image = image.img_to_array(test_image) / 255
  test_image = np.expand_dims(test_image, axis=0)

  predicted_array = model.predict(test_image)
  predicted_value = output_class[np.argmax(predicted_array)]
  predicted_accuracy = round(np.max(predicted_array) * 100, 2)

  print("Your waste material is ", predicted_value, " with ", predicted_accuracy, " % accuracy")

In [ ]:
waste_prediction("/content/drive/MyDrive/Garbage classification/glass/glass103.jpg")

In [ ]:
plt.title("Accuracy")
plt.plot(result.history["accuracy"])
plt.show()

In [ ]:
plt.title("Loss")
plt.plot(result.history["loss"])
plt.show()

In [28]:
model.save("classifyWaste.h5")